In [3]:
import pandas as pd
import os

# Define the cleaning function
def clean_weather_dataset(df):
    columns_to_drop = ['PGTM', 'SNOW', 'SNWD']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    if 'TAVG' in df.columns:
        df['TAVG'] = df['TAVG'].fillna((df['TMAX'] + df['TMIN']) / 2)

    numeric_cols = ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WSF2', 'WSF5']
    numeric_cols = [col for col in numeric_cols if col in df.columns]
    df[numeric_cols] = df[numeric_cols].interpolate(method='linear')

    wind_dir_cols = ['WDF2', 'WDF5']
    wind_dir_cols = [col for col in wind_dir_cols if col in df.columns]
    for col in wind_dir_cols:
        df[col] = df[col].ffill()

    wt_cols = ['WT01', 'WT02', 'WT03', 'WT05', 'WT08', 'WT13', 'WT16']
    wt_cols = [col for col in wt_cols if col in df.columns]
    df[wt_cols] = df[wt_cols].fillna(0)
    
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')

    float_cols = ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WSF2', 'WSF5']
    float_cols = [col for col in float_cols if col in df.columns]
    for col in float_cols:
        df[col] = pd.to_numeric(df[col].astype(str).str.strip(), errors='coerce', downcast='float')

    for col in wind_dir_cols:
        df[col] = pd.to_numeric(df[col].astype(str).str.strip(), errors='coerce').round().astype('Int32')

    for col in wt_cols:
        df[col] = df[col].astype('int32')

    if 'STATION' in df.columns:
        df['STATION'] = df['STATION'].astype(str)
    if 'NAME' in df.columns:
        df['NAME'] = df['NAME'].astype(str)

    for col in float_cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

    rename_dict = {
        'WT01': 'FOG',
        'WT02': 'HEAVY_FOG',
        'WT03': 'THUNDER',
        'WT05': 'HAIL',
        'WT08': 'SMOKE_HAZE',
        'WT13': 'MIST',
        'WT16': 'RAIN'
    }
    df = df.rename(columns={k: v for k, v in rename_dict.items() if k in df.columns})

    df = df.drop_duplicates()

    if 'TMIN' in df.columns and 'TAVG' in df.columns and 'TMAX' in df.columns:
        invalid_temp = (df['TMIN'] > df['TAVG']) | (df['TAVG'] > df['TMAX'])
        if invalid_temp.sum() > 0:
            print(f"Warning: {invalid_temp.sum()} rows with invalid temperature order")

    return df

file_paths = []
for year in range(2013, 2025):
    file_paths.append(f'https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_{year}.csv')

cleaned_dfs = []
for file_path in file_paths:
    try:
        print(f"Processing {file_path}...")
        df = pd.read_csv(file_path)  # Will raise error if file not found (404)
        cleaned_df = clean_weather_dataset(df)
        cleaned_dfs.append(cleaned_df)
        print(f"Successfully processed {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

if cleaned_dfs:
    combined_df = pd.concat(cleaned_dfs, ignore_index=True)
    print("Datasets combined successfully.")

    combined_df = combined_df.sort_values('DATE')

    duplicate_dates = combined_df.duplicated(subset=['DATE']).sum()
    if duplicate_dates > 0:
        print(f"Warning: {duplicate_dates} duplicate dates found. Removing duplicates...")
        combined_df = combined_df.drop_duplicates(subset=['DATE'], keep='first')

    print("Final dataset info:")
    print(combined_df.info())
    print(f"Date range: {combined_df['DATE'].min()} to {combined_df['DATE'].max()}")
else:
    print("No datasets were processed. Please check file paths.")

Processing https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2013.csv...
Successfully processed https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2013.csv
Processing https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2014.csv...
Successfully processed https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2014.csv
Processing https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2015.csv...
Successfully processed https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2015.csv
Processing https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weather_2016.csv...
Successfully processed https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/weather/raw_dataset/weath

In [4]:
combined_df

,STATION,NAME,DATE,AWND,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,FOG,HEAVY_FOG,THUNDER,HAIL,SMOKE_HAZE,MIST,RAIN
0,USW00023293,"SAN JOSE, CA US",2013-01-01,2.010000,0.0,45.0,55.0,35.0,360,20,8.9,16.100000,0,0,0,0.0,0,0.0,0.0
1,USW00023293,"SAN JOSE, CA US",2013-01-02,1.120000,0.0,44.5,58.0,31.0,260,160,6.9,8.100000,0,0,0,0.0,0,0.0,0.0
2,USW00023293,"SAN JOSE, CA US",2013-01-03,1.790000,0.0,45.0,57.0,33.0,310,310,8.1,8.900000,0,0,0,0.0,0,0.0,0.0
3,USW00023293,"SAN JOSE, CA US",2013-01-04,1.340000,0.0,47.5,60.0,35.0,90,50,6.9,8.900000,1,0,0,0.0,0,0.0,0.0
4,USW00023293,"SAN JOSE, CA US",2013-01-05,5.820000,0.0,48.0,56.0,40.0,90,100,17.0,21.000000,1,0,0,0.0,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4377,USW00023293,"SAN JOSE, CA US",2024-12-27,2.738065,0.0,60.0,66.0,54.0,290,280,12.1,14.358065,0,0,0,NaN,0,NaN,NaN
4378,USW00023293,"SAN JOSE, CA US",2024-12-28,2.723548,0.0,62.0,69.0,55.0,290,280,12.1,14.293549,0,0,0,NaN,0,NaN,NaN
4379,USW00023293,"SAN JOSE, CA US",2024-12-29,2.709032,0.0,57.0,65.0,49.0,290,280,12.1,14.229033,0,0,0,NaN,0,NaN,NaN
4380,USW00023293,"SAN JOSE, CA US",2024-12-30,2.694516,0.0,54.0,62.0,46.0,290,280,12.1,14.164516,0,0,0,NaN,0,NaN,NaN
